In [30]:
from datetime import datetime
from os.path import exists as file_exists
import pandas as pd
from pprint import pprint as pp
import re
import requests
from time import sleep
import xmltodict
import Credentials      # Get API keys, etc.

apikey = Credentials.prod_api
baseurl = 'https://api-na.hosted.exlibrisgroup.com'
item_query = '/almaws/v1/bibs/{mms_id}/holdings/{holding_id}/items/{item_pid}?apikey={apikey}'

exported_csv = "FullItemList.csv"
filled_csv = "FilledEnumChron.csv"
err_log_txt = "log.txt"

### Run this cell if you want to use the Limited-list CSV
exported_csv = "LimitedList.csv"

#### Create original df

In [112]:
# Depending on how the file was exported, column names may or may have either spaces or underscores
df = pd.read_csv(exported_csv, converters={'Item_ID': str, 'Holdings_ID': str, 'MMS_ID': str, 'Item ID': str, 'Holdings ID': str, 'MMS ID': str})

#### Clean up, tweak & format df

In [113]:
# Remove spaces from column names
df.columns = [c.replace(' ', '_') for c in df.columns]
# Rename certain columns
df = df.rename(columns={'Permanent_Location': 'Location', 'Item_Policy': 'Policy', 'Material_Type': 'Material'})
# Strip leading/trailing space from Description
df.Description = df.Description.str.strip()
# Collapse multiple spaces within the Description
df.Description.replace(' +', ' ', regex=True, inplace=True)

In [114]:
# Add columns for the Enum/Chron fields
EC_fields = ['Enum_A', 'Enum_B', 'Chron_I', 'Chron_J']
df[EC_fields] = None

#### The function for getting info from Description to Enum/Chron fields:

In [122]:
def fill_and_extract(regex, these_fields):
    exp = re.compile(regex)
    for i, f in enumerate(these_fields):
        df[f] = df['Description'].str.extract(exp, expand=True)[i].fillna(df[f])

#### Call it repeatedly for each regex that you come up with:
**[Test your Regex here](https://regex101.com/)**

In [124]:
# Just volume & nothing else
fill_and_extract(r'^v\.(\d)+$', ['Enum_A'])

In [125]:
# Just year or range of years (post-18th-century) & nothing else
fill_and_extract(r'^((?:18|19|20)\d{2})(?:[\-\/](?:19|20)\d{2})?)$', ['Chron_I'])

In [126]:
# Volume + year (or range of years)
fill_and_extract(r'^v\.(\d)+ ((?:19|20)\d{2}(?:[\-\/](?:19|20)?\d{2})?)$', ['Enum_A', 'Chron_I'])

#### Then once all those replacements are done, pull filled-in records out to a new dataframe

In [127]:
# Create a dataframe to hold JUST records that get filled
filled = pd.DataFrame()
# Populate the new dataframe with any records that now have at least one Enum/Chron field filled
filled = df.dropna(subset=EC_fields, thresh=1)

#### Then apply the changes via the API and log filled items to the Filled CSV

In [128]:
records = len(filled)
c = 0
needs_header=not file_exists(filled_csv) # Apparently we're creating the file, so it needs a header

In [129]:
### Run this bit to see what got "filled" before hitting the API

for index, row in filled.fillna('').iterrows():
    c += 1
    print(c, ' / '.join((row['MMS_ID'], row['Holdings_ID'], row['Item_ID'])), str(row['Description']), sep="\t")


1	991000134449706388 / 2276843580006388 / 2376843570006388	2000
2	991004057829706388 / 2260906600006388 / 2360906590006388	1970
3	991004332479706388 / 2266578400006388 / 2366578390006388	2010
4	991000847589706388 / 2271440170006388 / 2371440160006388	2017
5	991000843899706388 / 2269419430006388 / 2369419410006388	2009
6	991000843899706388 / 2269419430006388 / 2369419420006388	2009
7	991000847559706388 / 2271442340006388 / 2371442330006388	1993
8	991000430779706388 / 2273549730006388 / 2373549670006388	2011
9	991000843899706388 / 2269419400006388 / 2369419390006388	2009
10	991004404999706388 / 2260711870006388 / 2360711860006388	2021
11	991000847589706388 / 2271440150006388 / 2371440140006388	2017
12	991004219309706388 / 2274782130006388 / 2374782090006388	2011
13	991004219309706388 / 2274782130006388 / 2374782100006388	2010
14	991000081749706388 / 2268556360006388 / 2368556340006388	2000
15	991000115599706388 / 2269021040006388 / 2369021000006388	2003
16	991000523189706388 / 2276134730

625	991002786699706388 / 2266851610006388 / 2366851600006388	2009
626	991000751369706388 / 2272384760006388 / 2372384740006388	2001
627	991000767889706388 / 2274951480006388 / 2374951450006388	2003
628	991000767889706388 / 2274951480006388 / 2374951460006388	2002
629	991000767889706388 / 2274951480006388 / 2374951470006388	2010
630	991003302869706388 / 2262589460006388 / 2362589450006388	2009
631	991000248939706388 / 2266032260006388 / 2366032030006388	2005
632	991000248939706388 / 2266032260006388 / 2366032040006388	2004
633	991000248939706388 / 2266032260006388 / 2366032050006388	2002
634	991000248939706388 / 2266032260006388 / 2366032060006388	2001
635	991000248939706388 / 2266032260006388 / 2366032070006388	2000
636	991000248939706388 / 2266032260006388 / 2366032210006388	2010
637	991000248939706388 / 2266032260006388 / 2366032220006388	2009
638	991000248939706388 / 2266032260006388 / 2366032230006388	2008
639	991000248939706388 / 2266032260006388 / 2366032240006388	2007
640	991000

1170	991001313539706388 / 2266814880006388 / 2366814870006388	2004
1171	991004398159706388 / 2265675890006388 / 2365675810006388	2000
1172	991004230829706388 / 2275474070006388 / 2375474040006388	2018
1173	991004246279706388 / 2277008990006388 / 2377008860006388	2002
1174	991004246279706388 / 2277008990006388 / 2377008870006388	2001
1175	991004246279706388 / 2277008990006388 / 2377008890006388	2000
1176	991001220169706388 / 2268426300006388 / 2368426290006388	2004
1177	991002622709706388 / 2268595520006388 / 2368595500006388	2013
1178	991002622709706388 / 2268595520006388 / 2368595510006388	2009
1179	991004432549706388 / 2272612550006388 / 2372612520006388	2000
1180	991004320879706388 / 2270860820006388 / 2370860680006388	2004
1181	991004320879706388 / 2270860820006388 / 2370860690006388	2003
1182	991004527139706388 / 2265593600006388 / 2365593590006388	2000
1183	991001880329706388 / 2270725530006388 / 2370725520006388	2009
1184	991000620729706388 / 2270608510006388 / 2370608500006388	

1812	991001869849706388 / 2261176880006388 / 2361176860006388	1993
1813	991001214159706388 / 2273461960006388 / 2373461940006388	1995
1814	991001214159706388 / 2273461960006388 / 2373461950006388	1993
1815	991000446429706388 / 2269097960006388 / 2369097900006388	2004
1816	991000446429706388 / 2269097960006388 / 2369097910006388	2003
1817	991000446429706388 / 2269097960006388 / 2369097920006388	2002
1818	991000446429706388 / 2269097960006388 / 2369097930006388	2001
1819	991000446429706388 / 2269097960006388 / 2369097940006388	2000
1820	991000298079706388 / 2275160870006388 / 2375160830006388	2003
1821	991000298079706388 / 2275160870006388 / 2375160840006388	2001
1822	991000298079706388 / 2275160870006388 / 2375160850006388	2000
1823	991000298079706388 / 2275160870006388 / 2375160860006388	1998
1824	991004661619706388 / 2265465800006388 / 2365465720006388	1998
1825	991004661619706388 / 2265465800006388 / 2365465730006388	1997
1826	991004661619706388 / 2265465800006388 / 2365465740006388	

2337	991004661989706388 / 2265381820006388 / 2365381720006388	1982
2338	991004661989706388 / 2265381820006388 / 2365381730006388	1981
2339	991004661989706388 / 2265381820006388 / 2365381740006388	1980
2340	991004661989706388 / 2265381820006388 / 2365381750006388	1979
2341	991004661989706388 / 2265381820006388 / 2365381770006388	1978
2342	991004661989706388 / 2265381820006388 / 2365381780006388	1975
2343	991001150719706388 / 2274678250006388 / 2374678240006388	1982
2344	991004660909706388 / 2275116190006388 / 2375116140006388	1984
2345	991004660909706388 / 2275116190006388 / 2375116150006388	1983
2346	991004660909706388 / 2275116190006388 / 2375116170006388	1980
2347	991001602639706388 / 2276187460006388 / 2376187390006388	1900
2348	991001602639706388 / 2276187460006388 / 2376187420006388	1898
2349	991001602639706388 / 2276187460006388 / 2376187450006388	1896
2350	991001113229706388 / 2273834710006388 / 2373834630006388	1977
2351	991001113229706388 / 2273834710006388 / 2373834640006388	

...

30906	991002073569706388 / 2260708970006388 / 2360708720006388	v.120 2003
30907	991002073569706388 / 2260708970006388 / 2360708730006388	v.119 2002
30908	991002073569706388 / 2260708970006388 / 2360708770006388	v.118 2001
30909	991002073569706388 / 2260708970006388 / 2360708790006388	v.117 2000
30910	991002073569706388 / 2260708970006388 / 2360708820006388	v.116 1999
30911	991002073569706388 / 2260708970006388 / 2360708830006388	v.115 1998
30912	991002073569706388 / 2260708970006388 / 2360708850006388	v.114 1997
30913	991002073569706388 / 2260708970006388 / 2360708870006388	v.113 1996
30914	991002073569706388 / 2260708970006388 / 2360708950006388	v.124 2007
30915	991002073569706388 / 2260708970006388 / 2360708960006388	v.112 1995
30916	991003092149706388 / 2274209730006388 / 2374207850006388	v.17 1941/42
30917	991003092149706388 / 2274209730006388 / 2374207860006388	v.16 1940/41
30918	991003092149706388 / 2274209730006388 / 2374207870006388	v.15 1939/40
30919	991003092149706388 / 22742

31427	991003091909706388 / 2260617160006388 / 2360616140006388	v.23 1987
31428	991003091909706388 / 2260617160006388 / 2360616150006388	v.22 1986
31429	991003091909706388 / 2260617160006388 / 2360616160006388	v.21 1985
31430	991003091909706388 / 2260617160006388 / 2360616170006388	v.20 1984
31431	991003091909706388 / 2260617160006388 / 2360616180006388	v.19 1983
31432	991003091909706388 / 2260617160006388 / 2360616190006388	v.18 1982
31433	991003091909706388 / 2260617160006388 / 2360616640006388	v.34 1998
31434	991001601759706388 / 2264374620006388 / 2364372280006388	v.115 2006/07
31435	991001601759706388 / 2264374620006388 / 2364372390006388	v.114 2005/06
31436	991001601759706388 / 2264374620006388 / 2364372420006388	v.98 1989
31437	991001601759706388 / 2264374620006388 / 2364372430006388	v.94 1985
31438	991001601759706388 / 2264374620006388 / 2364372440006388	v.93 1984
31439	991001601759706388 / 2264374620006388 / 2364372450006388	v.92 1983
31440	991001601759706388 / 2264374620006388

31989	991000641149706388 / 2260840770006388 / 2360840250006388	v.69 1999
31990	991000641149706388 / 2260840770006388 / 2360840260006388	v.68 1998
31991	991000641149706388 / 2260840770006388 / 2360840270006388	v.67 1997
31992	991000641149706388 / 2260840770006388 / 2360840360006388	v.66 1996
31993	991000641149706388 / 2260840770006388 / 2360840650006388	v.65 1995
31994	991000641149706388 / 2260840770006388 / 2360840680006388	v.77 2007
31995	991000641149706388 / 2260840770006388 / 2360840760006388	v.64 1994
31996	991000641149706388 / 2260840770006388 / 2360859410006388	v.76 2006
31997	991000641149706388 / 2260840770006388 / 2360859420006388	v.75 2005
31998	991000641149706388 / 2260840770006388 / 2360859430006388	v.52 1982
31999	991000641149706388 / 2260840770006388 / 2360859440006388	v.51 1981
32000	991000641149706388 / 2260840770006388 / 2360859450006388	v.50 1980
32001	991002073569706388 / 2260708970006388 / 2360708130006388	v.123 2006
32002	991002073569706388 / 2260708970006388 / 2360

32660	991003825799706388 / 2260825510006388 / 2360824870006388	v.125 1920
32661	991003825799706388 / 2260825510006388 / 2360824880006388	v.124 1919
32662	991003825799706388 / 2260825510006388 / 2360824890006388	v.123 1919
32663	991003825799706388 / 2260825510006388 / 2360824900006388	v.122 1918
32664	991003825799706388 / 2260825510006388 / 2360824910006388	v.121 1918
32665	991003825799706388 / 2260825510006388 / 2360824930006388	v.120 1917
32666	991003825799706388 / 2260825510006388 / 2360824960006388	v.119 1917
32667	991003825799706388 / 2260825510006388 / 2360824970006388	v.118 1916
32668	991003825799706388 / 2260825510006388 / 2360824980006388	v.117 1916
32669	991003825799706388 / 2260825510006388 / 2360824990006388	v.116 1915
32670	991003825799706388 / 2260825510006388 / 2360825000006388	v.115 1915
32671	991003825799706388 / 2260825510006388 / 2360825010006388	v.114 1914
32672	991003825799706388 / 2260825510006388 / 2360825020006388	v.113 1914
32673	991003825799706388 / 22608255100

33241	991003961729706388 / 2270524760006388 / 2370524740006388	v.2
33242	991003961729706388 / 2270524760006388 / 2370524750006388	v.1
33243	991000888849706388 / 2272153280006388 / 2372153260006388	v.1
33244	991002927319706388 / 2271573180006388 / 2371573150006388	v.4
33245	991002927319706388 / 2271573180006388 / 2371573160006388	v.1
33246	991002927319706388 / 2271573180006388 / 2371573170006388	v.1
33247	991001042809706388 / 2262542270006388 / 2362542150006388	v.4
33248	991001042809706388 / 2262542270006388 / 2362542160006388	v.3
33249	991001042809706388 / 2262542270006388 / 2362542170006388	v.6
33250	991001042809706388 / 2262542270006388 / 2362542220006388	v.2
33251	991001042809706388 / 2262542270006388 / 2362542230006388	v.1
33252	991001042809706388 / 2262542270006388 / 2362542240006388	v.5
33253	991001042809706388 / 2262542270006388 / 2362542250006388	v.2
33254	991001042809706388 / 2262542270006388 / 2362542260006388	v.1
33255	991000693019706388 / 2260927150006388 / 2360927130006388

In [ ]:
with open(err_log_txt, 'a') as err_log:
    for index, row in filled.fillna('').iterrows():
        c += 1
        print(c, '/'.join((row['MMS_ID'], row['Holdings_ID'], row['Item_ID'])), str(row['Description']), sep="\t")
        r = requests.get(''.join([baseurl,
                                  item_query.format(mms_id=str(row['MMS_ID']),
                                                    holding_id=str(row['Holdings_ID']),
                                                    item_pid=str(row['Item_ID']),
                                                    apikey=apikey)]))
        rdict = xmltodict.parse(r.text)
        if r.status_code != 200:
            e = xmltodict.parse(r._content)
            # Log the error
            print(datetime.now().strftime("%Y-%m-%d %H:%M:%S"), ' Error FETCHING item ', row['Item_ID'], ': (', r.status_code, ') ',
                e['web_service_result']['errorList']['error']['errorMessage'],
                sep='', file=err_log)
            # Remove this item from the "filled" df
            filled = filled.drop([index])
            continue
        if (c % (records/100) < 1):
            print(int(100*c/records), '% complete', sep='')#, end='\r')
            sleep(5)
            
        # Merge derived values into the retrieved data (rdict)
        rdict['item']['item_data']['enumeration_a'] = str(row['Enum_A'])
        rdict['item']['item_data']['enumeration_b'] = str(row['Enum_B'])
        rdict['item']['item_data']['chronology_i'] = str(row['Chron_I'])
        rdict['item']['item_data']['chronology_j'] = str(row['Chron_J'])
        # Set an internal note, if there's an empty one available
        if ('Enum/Chron derived from Description' not in rdict['item']['item_data'].values()):
            if (not rdict['item']['item_data']['internal_note_1']):
                rdict['item']['item_data']['internal_note_1'] = 'Enum/Chron derived from Description'
            elif (not rdict['item']['item_data']['internal_note_2']):
                rdict['item']['item_data']['internal_note_2'] = 'Enum/Chron derived from Description'
            elif (not rdict['item']['item_data']['internal_note_3']):
                rdict['item']['item_data']['internal_note_3'] = 'Enum/Chron derived from Description'
            else: # Nbd, just log it
                print(datetime.now().strftime("%Y-%m-%d %H:%M:%S"), ' No internal note available for item MMS ID ',
                    str(row['MMS_ID']), sep="", file=err_log)
 
        # Push the altered record back into Alma
        pxml = xmltodict.unparse(rdict)
        p = requests.put(''.join([baseurl,
                                  item_query.format(mms_id=row['MMS_ID'],
                                                    holding_id=row['Holdings_ID'],
                                                    item_pid=row['Item_ID'],
                                                    apikey=apikey)]),
                         data=pxml.encode('utf-8'), headers={'Content-Type': 'application/xml'})
        if p.status_code != 200:
            e = xmltodict.parse(p._content)
            # Log the error
            print(datetime.now().strftime("%Y-%m-%d %H:%M:%S"), ' Error UPDATING item ', row['Item_ID'], ': (', p.status_code, ') ',
                e['web_service_result']['errorList']['error']['errorMessage'],
                sep='', file=err_log)
            # Remove this item from the "filled" df
            filled = filled.drop([index])
            continue

        # Log it to the CSV
        #    Btw, the 'to_frame().T' transposes it, so it all goes in as a single comma-separated row
        row.to_frame().T.to_csv(filled_csv, mode='a', index=False, header=needs_header)
        needs_header=False # Henceforth

#### Purge filled rows from the original CSV

In [53]:
# Purge filled records from the original df
df = df.loc[~df['Item_ID'].isin(filled['Item_ID'])]

# Re-create the original CSV from that df
df.to_csv(exported_csv, index=False) # By default, will overwrite

# View df & filled

In [ ]:
display(df)
display(filled)

# Testing stuff